<a href="https://colab.research.google.com/github/hj56775/dlibforbeauty/blob/main/%EC%98%A4%ED%86%A0%EC%9D%B8%EC%BD%94%EB%8D%94.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pwd

/content


In [2]:
%cd drive/MyDrive/Colab Notebooks

/content/drive/MyDrive/Colab Notebooks


In [3]:
!ls

callbacks.py			     __pycache__
DailyDelhiClimateTest.csv	     study_cnn.ipynb
DailyDelhiClimateTrain.csv	     Untitled
dlib_beauty.ipynb		     Untitled0.ipynb
images				     Untitled1.ipynb
MLPDelhiClimateAnalysis.ipynb	     Untitled2.ipynb
OBS_ASOS_DD_20201207220648.csv	     Untitled3.ipynb
OBS_ASOS_DD_20201207220648_TEST.csv


In [4]:
import os

In [5]:
import numpy as np
import dlib
import cv2
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from google.colab.patches import cv2_imshow
from callbacks import CustomCallback, step_decay_schedule

In [6]:
from tensorflow import keras
mnist = keras.datasets.mnist

(X_train,y_train), (X_test, y_test) = mnist.load_data()

In [7]:
from keras.layers import Input, Conv2D, Flatten, Dense, Conv2DTranspose, Reshape, Lambda, Activation, BatchNormalization, LeakyReLU, Dropout
from keras.models import Model
from keras import backend as K
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint 
from keras.utils import plot_model 
import tensorflow as tf
import numpy as np

input_dim=(28,28,1)
filter=[32,64,64,64]
stride=[1,2,2,1]
encoder_input = Input(shape=input_dim, name='input')

x = encoder_input

for i in range(4):
    conv_layer = Conv2D(
        filters = filter[i]
        , kernel_size = 3
        , strides = stride[i]
        ,padding = 'same'
        , name = 'conv' + str(i)
        )

    x = conv_layer(x)
    x = LeakyReLU()(x)


shape_before_flattening = K.int_shape(x)[1:]

x = Flatten()(x)
encoder_output=Dense(2,name='encoder_output')(x)


encoder=Model(encoder_input,encoder_output)
encoder.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           [(None, 28, 28, 1)]       0         
_________________________________________________________________
conv0 (Conv2D)               (None, 28, 28, 32)        320       
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 28, 28, 32)        0         
_________________________________________________________________
conv1 (Conv2D)               (None, 14, 14, 64)        18496     
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 14, 14, 64)        0         
_________________________________________________________________
conv2 (Conv2D)               (None, 7, 7, 64)          36928     
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 7, 7, 64)          0     

In [8]:
shape_before_flattening

(7, 7, 64)

In [9]:
filter=[64,64,32,1]
stride=[1,2,2,1]
decoder_input = Input(shape=2, name='decoder_input')

x = Dense(np.prod(shape_before_flattening))(decoder_input)
x=Reshape(shape_before_flattening)(x)

for i in range(4):
    conv_t_layer = Conv2DTranspose(
        filters = filter[i]
        , kernel_size = 3
        , strides = stride[i]
        ,padding = 'same'
        , name = 'decoder_conv_t' + str(i)
        )

    x = conv_t_layer(x)

    if i <3:
      x=LeakyReLU()(x)
    else:
      x=Activation('sigmoid')(x)

decoder_output=x


decoder=Model(decoder_input,decoder_output)
decoder.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
decoder_input (InputLayer)   [(None, 2)]               0         
_________________________________________________________________
dense (Dense)                (None, 3136)              9408      
_________________________________________________________________
reshape (Reshape)            (None, 7, 7, 64)          0         
_________________________________________________________________
decoder_conv_t0 (Conv2DTrans (None, 7, 7, 64)          36928     
_________________________________________________________________
leaky_re_lu_4 (LeakyReLU)    (None, 7, 7, 64)          0         
_________________________________________________________________
decoder_conv_t1 (Conv2DTrans (None, 14, 14, 64)        36928     
_________________________________________________________________
leaky_re_lu_5 (LeakyReLU)    (None, 14, 14, 64)        0   

In [10]:
#오토인코더
model_input=encoder_input
model_output=decoder(encoder_output)
model=Model(model_input,model_output)

In [11]:
model.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           [(None, 28, 28, 1)]       0         
_________________________________________________________________
conv0 (Conv2D)               (None, 28, 28, 32)        320       
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 28, 28, 32)        0         
_________________________________________________________________
conv1 (Conv2D)               (None, 14, 14, 64)        18496     
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 14, 14, 64)        0         
_________________________________________________________________
conv2 (Conv2D)               (None, 7, 7, 64)          36928     
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 7, 7, 64)          0   

In [12]:
lr_sched = step_decay_schedule(initial_lr=0.0005, decay_factor=1, step_size=1)

In [13]:
custom_callback = CustomCallback("/content/drive/MyDrive/Colab Notebooks", 100, 0,decoder)

In [14]:
checkpoint2 = ModelCheckpoint(os.path.join("/content/drive/MyDrive/Colab Notebooks", 'weights/weights.h5'), save_weights_only = True, verbose=1)

In [15]:
callbacks_list = [checkpoint2, custom_callback, lr_sched]

In [16]:
#컴파일
model.compile(optimizer=Adam(lr=0.0005),loss= 'mse')

In [18]:
model.fit(   
    X_train
    , X_train
    , batch_size =32
    , shuffle = True
    , epochs = 10
    , initial_epoch = 0
    , callbacks = callbacks_list
)

Epoch 1/10
1875/1875 [==============================] - 8s 4ms/step - loss: 7217.1670

Epoch 00001: saving model to /content/drive/MyDrive/Colab Notebooks/weights/weights.h5
Epoch 2/10
1875/1875 [==============================] - 8s 4ms/step - loss: 7217.1670

Epoch 00002: saving model to /content/drive/MyDrive/Colab Notebooks/weights/weights.h5
Epoch 3/10
1875/1875 [==============================] - 8s 4ms/step - loss: 7217.1572

Epoch 00003: saving model to /content/drive/MyDrive/Colab Notebooks/weights/weights.h5
Epoch 4/10
1875/1875 [==============================] - 8s 4ms/step - loss: 7217.1392

Epoch 00004: saving model to /content/drive/MyDrive/Colab Notebooks/weights/weights.h5
Epoch 5/10
1875/1875 [==============================] - 8s 4ms/step - loss: 7217.1523

Epoch 00005: saving model to /content/drive/MyDrive/Colab Notebooks/weights/weights.h5
Epoch 6/10
1875/1875 [==============================] - 8s 4ms/step - loss: 7217.1411

Epoch 00006: saving model to /content/drive

In [ ]:
model.decoder